# Two Phase method


In [143]:
import numpy as np
import pandas as pd

Infeasible = False

### Reading the problem from csv file

required format for the .csv file

   **max/min,  x1,  x2, x3, ...**
   
   **xc11, xc12, xc13, ... <=/>=/= b1**

   **xc21, xc22, xc23, ... <=/>=/= b2**

        ...     ...     ...



In [144]:


problem = pd.read_csv('twoPhase.csv', header=None) 

optimization_function = problem.iloc[0].dropna() # extract the optimization function
optimization_function = optimization_function.to_numpy()
# headers = optimization_function.to_frame().iloc[0]
# optimization_function  = pd.DataFrame(optimization_function.values[1:], columns=headers)

# convert the optimization function to a maximization problem
if optimization_function[0] == 'max':
    optimization_function =  optimization_function[1:].astype(float)
else:   
    optimization_function =-1 * optimization_function[1:].astype(float)


# constraints=problem.iloc[[i for i in range(1,problem.shape[0])]] # extract the constraints
constraints = problem.iloc[1:].dropna() # extract the constraints

print("Extracted problem from the csv file is->\n")

print("\nMaximize -> " , optimization_function)
print( "\nConstarints are ->\n",constraints.to_string(index=False, header=False))
print(constraints.shape[0])



Extracted problem from the csv file is->


Maximize ->  [-1. -1.]

Constarints are ->
 2 1 >= 4.0
1 7 >= 7.0
2



### Introducing Slack Variables, Surplus and Artificial variables



In [145]:

# constraints = constraints.to_numpy()
b = constraints.iloc[:,-1:]

constraints_coefficients = constraints.iloc[:,:-2]
constraints_coefficients = constraints_coefficients.to_numpy().astype(float)

artificial_variables_position = []

# add slack variables to the constraints
for i in range(constraints.shape[0]):
    if(constraints.iloc[i,-2] == '<='):
        constraints_coefficients = np.hstack((constraints_coefficients, np.zeros((constraints.shape[0],1))))
        constraints_coefficients[i][-1] = 1

print("Slack variables are added to the constraints ->\n")
print(constraints_coefficients)

# add surplus and artificial variables to the constraints
for i in range(constraints.shape[0]):
    if(constraints.iloc[i,-2] == '>='):
        constraints_coefficients = np.hstack((constraints_coefficients, np.zeros((constraints.shape[0],1))))
        constraints_coefficients[i][-1] = -1
        constraints_coefficients = np.hstack((constraints_coefficients, np.zeros((constraints.shape[0],1))))
        constraints_coefficients[i][-1] = 1
        # add the position of the artificial variable
        artificial_variables_position.append(constraints_coefficients.shape[1]-1)



print("Surplus variables are added to the constraints ->\n")
print(constraints_coefficients)

# add artificial variables to the constraints
for i in range(constraints.shape[0]):
    if(constraints.iloc[i,-2] == '='):
        constraints_coefficients = np.hstack((constraints_coefficients, np.zeros((constraints.shape[0],1))))
        constraints_coefficients[i][-1] = 1
        # add the position of the artificial variable
        artificial_variables_position.append(constraints_coefficients.shape[1]-1)

print("Artificial variables are added to the constraints ->\n")
print(constraints_coefficients)


b=b.to_numpy()
b = b.astype(float)

c_j = np.append(optimization_function, np.zeros(constraints_coefficients.shape[1]-optimization_function.shape[0]))

c_B = np.zeros(constraints_coefficients.shape[0])
c_B = c_B.astype(float)


# assign bigM to the artificial variables cost
for i in artificial_variables_position:
    c_j[i] = -1

#calculate c_B
for i in range(constraints_coefficients.shape[0]):
    for j in range(optimization_function.shape[0],constraints_coefficients.shape[1]):
        if constraints_coefficients[i][j] == 1:
            # print("i ->",i,"j ->",j)
            c_B[i] = c_j[j]
            # print("c_j",[j], "->",c_j[j])
            # print("c_B ->",c_B)

c_B = c_B.reshape(-1,1)

print("c_j ->\n",c_j) 

# for i in range(c_j.shape[0]):
#     print("c_j[",i,"] ->",c_j[i])

print("\n\nThe problem is converted to the following standard form ->\n")
print("\nc_j ->\n",c_j)
print("\nb ->\n",b)
print("\nconstraints ->\n",constraints_coefficients)

# create the initial table
table = np.hstack((b,constraints_coefficients))


print("\n\nThe initial table is ->\n")
print(c_B, table)



Slack variables are added to the constraints ->

[[2. 1.]
 [1. 7.]]
Surplus variables are added to the constraints ->

[[ 2.  1. -1.  1.  0.  0.]
 [ 1.  7.  0.  0. -1.  1.]]
Artificial variables are added to the constraints ->

[[ 2.  1. -1.  1.  0.  0.]
 [ 1.  7.  0.  0. -1.  1.]]
c_j ->
 [-1. -1.  0. -1.  0. -1.]


The problem is converted to the following standard form ->


c_j ->
 [-1. -1.  0. -1.  0. -1.]

b ->
 [[4.]
 [7.]]

constraints ->
 [[ 2.  1. -1.  1.  0.  0.]
 [ 1.  7.  0.  0. -1.  1.]]


The initial table is ->

[[-1.]
 [-1.]] [[ 4.  2.  1. -1.  1.  0.  0.]
 [ 7.  1.  7.  0.  0. -1.  1.]]



### Phase-I iterations


In [146]:

# b_order = np.arange(constraints.shape[1]+1, table.shape[1])
# print("\n\nb_order is ->\n",b_order)

# assigning 0 costs to the optimization function
for i in range(optimization_function.shape[0]):
    c_j[i] = 0

print("current c_j ->\n",c_j)

print("c_B ->\n",c_B)


for i in range(100):
    # find the initial basic feasible solution
    z_j = np.zeros(table.shape[1]-1)
    for i in range(c_B.shape[0]):
        z_j += c_B[i] * np.delete(table, 0, 1)[i]
        print(c_B[i], "*", np.delete(table, 0, 1)[i])
    print("\n\nz_j is ->\n",z_j)


    # find the initial z_j - c_j
    z_j_c_j = z_j - c_j
    print("\n\n z_j - c_j is \n",z_j_c_j)

    

    if (z_j_c_j >= 0).all():
        print("\n\nThe solution is optimal")
        b=table[:,0]
        print("optimal table is ->\n",c_B,table)
        break
    else:
        incoming_variable_index = np.argmin(z_j_c_j)
        print("\nIcoming variable's index = ", incoming_variable_index+1,"\nCounting min ratio...")
        # find the min ratio
        min_ratio = np.inf
        for i in range(table.shape[0]):
            if table[i][incoming_variable_index+1] > 0:
                ratio = table[i][0]/table[i][incoming_variable_index+1]
                print("Ratio is ->", table[i][0],"/",table[i][incoming_variable_index+1],"=",ratio)
                if ratio < min_ratio:
                    if ratio <0:
                        print("Ratio is negative")
                        continue
                    min_ratio = ratio
                    outgoing_variable_index = i
        if min_ratio == np.inf:
            print("Unbounded solution")
            break
        print("\nOutgoing variable's index = ", outgoing_variable_index)

        print("\nThe key value is ->\n",table[outgoing_variable_index][incoming_variable_index+1])

        # delete the column of artificial variable
        for i in artificial_variables_position:
            print("i ->",i)
            if c_B[outgoing_variable_index] == c_j[i]:
                print("\nArtificial variable is going out", c_B[outgoing_variable_index])
                artificial_variables_position.remove(i)
                # artificial_variables_position = artificial_variables_position - 1
                artificial_variables_position = [ x-1 for x in artificial_variables_position if x>=len(artificial_variables_position)]
                print("Artificial variables position ->",artificial_variables_position)
                table = np.delete(table, i+1, 1)
                c_j = np.delete(c_j, i)
                print("current c_j ->",c_j)
                print("\n\nThe table after", i,"th col delete ->\n",table)
        # c_B[outgoing_variable_index] = table[outgoing_variable_index][incoming_variable_index+1]
        c_B[outgoing_variable_index] = c_j[incoming_variable_index]
        print("\nc_B is ->\n",c_B)

        
                # print(c_B,table) 
            # print(c_B,table)
        # b_order[outgoing_variable_index] = incoming_variable_index+1

        # update the table
        # z_j = np.zeros(table.shape[1]-1)

        table[outgoing_variable_index] = table[outgoing_variable_index]/table[outgoing_variable_index][incoming_variable_index+1]
        for i in range(table.shape[0]):
            if i != outgoing_variable_index:
                table[i] = table[i] - table[outgoing_variable_index]*table[i][incoming_variable_index+1]
        print("\n\nThe updated table is ->\n")
        print(c_B,table)

# print(b_order)

# find the optimal solution
# maxZ = 0
# for i in range(b_order.shape[0]):
#     print("x",b_order[i],"=",b[i])
#     maxZ += optimization_function[b_order[i]-1]*b[i]

for i in range(b.shape[0]):
    if b[i] < 0:
        print("Infeasible solution")
        Infeasible = True
    elif c_B[i]>0:
        print("Infeasible solution")
        Unbounded = True

# maxZ = 0
# for i in range(c_B.shape[0]):
#     print(c_B[i],"*",b[i])
#     maxZ += c_B[i]*b[i]

# print("Max Z = ",maxZ)





current c_j ->
 [ 0.  0.  0. -1.  0. -1.]
c_B ->
 [[-1.]
 [-1.]]
[-1.] * [ 2.  1. -1.  1.  0.  0.]
[-1.] * [ 1.  7.  0.  0. -1.  1.]


z_j is ->
 [-3. -8.  1. -1.  1. -1.]


 z_j - c_j is 
 [-3. -8.  1.  0.  1.  0.]

Icoming variable's index =  2 
Counting min ratio...
Ratio is -> 4.0 / 1.0 = 4.0
Ratio is -> 7.0 / 7.0 = 1.0

Outgoing variable's index =  1

The key value is ->
 7.0
i -> 3

Artificial variable is going out [-1.]
Artificial variables position -> [4]
current c_j -> [ 0.  0.  0.  0. -1.]


The table after 3 th col delete ->
 [[ 4.  2.  1. -1.  0.  0.]
 [ 7.  1.  7.  0. -1.  1.]]

c_B is ->
 [[-1.]
 [ 0.]]


The updated table is ->

[[-1.]
 [ 0.]] [[ 3.          1.85714286  0.         -1.          0.14285714 -0.14285714]
 [ 1.          0.14285714  1.          0.         -0.14285714  0.14285714]]
[-1.] * [ 1.85714286  0.         -1.          0.14285714 -0.14285714]
[0.] * [ 0.14285714  1.          0.         -0.14285714  0.14285714]


z_j is ->
 [-1.85714286  0.          1.  

### Phase-II iterations

In [147]:

# b_order = np.arange(constraints.shape[1]+1, table.shape[1])
# print("\n\nb_order is ->\n",b_order)

# assign oringinal costs to the optimization function
for i in range(optimization_function.shape[0]):
    c_j[i] = optimization_function[i]
print("current c_j ->\n",c_j)

print("current constraints_coefficients ->\n",constraints_coefficients)

# calculate current c_B for phase II
for i in range(constraints_coefficients.shape[0]):
    for j in range(optimization_function.shape[0]):
        if constraints_coefficients[i][j] == 1:
            print("c_B[",i,"] = c_j[",j,"]")
            c_B[i] = c_j[j]

print("current c_B ->\n",c_B)




for i in range(100):
    # find the initial basic feasible solution
    z_j = np.zeros(table.shape[1]-1)
    for i in range(c_B.shape[0]):
        z_j += c_B[i] * np.delete(table, 0, 1)[i]
        print(c_B[i], "*", np.delete(table, 0, 1)[i])
    print("\n\nz_j is ->\n",z_j)


    # find the initial z_j - c_j
    z_j_c_j = z_j - c_j
    print("\n\n z_j - c_j is \n",z_j_c_j)

    

    if (z_j_c_j >= 0).all():
        print("\n\nThe solution is optimal")
        b=table[:,0]
        break
    else:
        incoming_variable_index = np.argmin(z_j_c_j)
        print("\nIcoming variable's index = ", incoming_variable_index+1,"\nCounting min ratio...")
        # find the min ratio
        min_ratio = np.inf
        for i in range(table.shape[0]):
            if table[i][incoming_variable_index+1] > 0:
                ratio = table[i][0]/table[i][incoming_variable_index+1]
                print("Ratio is ->", table[i][0],"/",table[i][incoming_variable_index+1],"=",ratio)
                if ratio < min_ratio:
                    if ratio <0:
                        print("Ratio is negative")
                        continue
                    min_ratio = ratio
                    outgoing_variable_index = i
        if min_ratio == np.inf:
            print("Unbounded solution")
            break
        print("\nOutgoing variable's index = ", outgoing_variable_index)

        print("\nThe key value is ->\n",table[outgoing_variable_index][incoming_variable_index+1])

        # c_B[outgoing_variable_index] = table[outgoing_variable_index][incoming_variable_index+1]
        c_B[outgoing_variable_index] = c_j[incoming_variable_index]
        print("\nc_B is ->\n",c_B)

        
            
            # print(c_B,table)
        # b_order[outgoing_variable_index] = incoming_variable_index+1

        # update the table
        # z_j = np.zeros(table.shape[1]-1)

        table[outgoing_variable_index] = table[outgoing_variable_index]/table[outgoing_variable_index][incoming_variable_index+1]
        for i in range(table.shape[0]):
            if i != outgoing_variable_index:
                table[i] = table[i] - table[outgoing_variable_index]*table[i][incoming_variable_index+1]
        print("\n\nThe updated table is ->\n")
        print(c_B,table)

for i in range(b.shape[0]):
    if b[i] < 0:
        print("Infeasible solution")
        Infeasible = True

maxZ = 0
for i in range(c_B.shape[0]):
    print(c_B[i],"*",b[i])
    maxZ += c_B[i]*b[i]

print("Max Z = ",maxZ)

current c_j ->
 [-1. -1.  0.  0.]
current constraints_coefficients ->
 [[ 2.  1. -1.  1.  0.  0.]
 [ 1.  7.  0.  0. -1.  1.]]
c_B[ 0 ] = c_j[ 1 ]
c_B[ 1 ] = c_j[ 0 ]
current c_B ->
 [[-1.]
 [-1.]]
[-1.] * [ 1.          0.         -0.53846154  0.07692308]
[-1.] * [ 0.          1.          0.07692308 -0.15384615]


z_j is ->
 [-1.         -1.          0.46153846  0.07692308]


 z_j - c_j is 
 [0.         0.         0.46153846 0.07692308]


The solution is optimal
[-1.] * 1.6153846153846154
[-1.] * 0.7692307692307693
Max Z =  [-2.38461538]
